# Femnist Model Testing

Goals:
- Load Data from data folder
- Process Data for model
- Run Training for some Generations

## Load Data

In [13]:
from pathlib import Path
import os
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
base_directory = Path(os.getcwd()).parent.parent
femnist_data_root_directory = base_directory / "data" / "femnist"

sample = 0
femnist_data_sample_directory = femnist_data_root_directory / f"{sample}"

femnist_sample_test_file = femnist_data_sample_directory / "test.pickle"
femnist_sample_train_file = femnist_data_sample_directory / "train.pickle"
femnist_sample_validation_file = femnist_data_sample_directory / "val.pickle"

In [15]:
femnist_sample_train_data = pd.read_pickle(femnist_sample_train_file)
femnist_sample_test_data = pd.read_pickle(femnist_sample_test_file)
femnist_sample_validation_data = pd.read_pickle(femnist_sample_validation_file)

## Create Model

In [16]:
from sources.models.femnist.femnist_client_model import FemnistClientModel

client_model = FemnistClientModel(0.1)
model = client_model.get_model()

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3136)              0         
                                                                 
 dense_2 (Dense)             (None, 2048)             

## Process Data, Compile Model and Fit

In [18]:
client_model.process_x(femnist_sample_train_data["x"]).shape

(207, 28, 28)

In [19]:
model.compile(optimizer=client_model.get_default_optimizer(),
             loss=client_model.get_default_loss_function(),
             metrics=['accuracy', "mse"])

len(femnist_sample_train_data["x"])

207

In [20]:
model.fit(client_model.process_x(femnist_sample_train_data["x"]),
          client_model.process_y(femnist_sample_train_data["y"]),
          validation_data= (client_model.process_x(femnist_sample_test_data["x"]),
                            client_model.process_y(femnist_sample_test_data["y"])),
          epochs=1)

7/7 [==============================] - 1s 55ms/step - loss: 11613872586752.0000 - accuracy: 0.0483 - mse: 908.4978 - val_loss: 4.1381 - val_accuracy: 0.0714 - val_mse: 735.7827


In [21]:
model.evaluate(
    x=client_model.process_x(femnist_sample_validation_data["x"]),
    y=client_model.process_y(femnist_sample_validation_data["y"]),
)

3/3 [==============================] - 0s 7ms/step - loss: 5.0514 - accuracy: 0.0000e+00 - mse: 25.0970


[5.051393032073975, 0.0, 25.096986770629883]